In [1]:
data_var = '2024-01-14'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12371,2024-01-14,Libano Divisão 1,11:45,Antonine,Al-Riyadi,5.03,1.15,162.5,1.84,1.86,10.5,2.03,1.70,IaJb2WOt,0.198807,0.869565,0.543478,0.537634,0.068372,183.644,72.298018,0.393686,2.4,1.341641,0.559017,301.60,2.356,0.934173,0.396508,18.0,106.700,7.774098,0.072859,2.4,1.341641,0.559017,108.16,1.236,0.223338,0.180694,46.0,80,104,3.77,1.04,0.000,0.000,0.887888,0.007644,0.125118,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
12372,2024-01-14,Lituânia Lkl,12:20,Siauliai,Zalgiris Kaunas,5.19,1.14,160.5,1.86,1.84,10.5,2.01,1.71,QZmTUU2I,0.192678,0.877193,0.537634,0.543478,0.069871,257.314,160.940853,0.625465,1.8,1.643168,0.912871,250.70,3.138,2.199289,0.700857,7.0,96.948,16.484279,0.170032,2.4,1.341641,0.559017,83.25,1.182,0.188865,0.159784,36.0,115,75,2.18,1.11,0.000,0.000,0.904829,0.007644,0.114049,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
12373,2024-01-14,Lituânia Lkl,14:30,Neptunas,Gargzdai,1.16,4.71,168.5,1.81,1.85,-12.5,2.05,1.69,vwhXTlIO,0.862069,0.212314,0.552486,0.540541,0.074383,137.240,44.225285,0.322248,1.2,1.643168,1.369306,166.14,1.858,0.625516,0.336661,-14.0,307.030,263.503363,0.858233,0.0,0.000000,NaN,195.98,3.900,3.336113,0.855414,-70.0,78,82,2.13,2.39,0.000,0.000,0.855274,0.015456,0.136128,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
12374,2024-01-14,Itália Série A1 Feminina,14:00,Sanga Milano F,San Martino F,6.47,1.09,138.5,1.85,1.85,11.5,2.07,1.67,MJ0Q19vN,0.154560,0.917431,0.540541,0.540541,0.071991,633.380,304.591878,0.480899,0.0,0.000000,NaN,568.40,11.660,4.991793,0.428113,-127.0,231.212,178.908419,0.773785,1.2,1.643168,1.369306,236.50,3.448,2.746793,0.796634,4.0,49,55,11.60,4.30,0.000,0.000,1.006411,0.000000,0.151253,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.000000,0.0,0.000000
12375,2024-01-14,Noruega Blno,11:00,Nidaros Jets,Kongsberg Miners,5.56,1.12,159.5,1.83,1.87,10.5,2.05,1.65,0ddwt7ni,0.179856,0.892857,0.546448,0.534759,0.072713,151.284,58.237150,0.384952,0.6,1.341641,2.236068,248.98,2.260,1.143044,0.505772,-59.0,194.584,197.129382,1.013081,1.8,1.643168,0.912871,112.80,2.204,2.329534,1.056957,62.0,59,94,4.22,1.20,0.000,108.620,0.939986,0.015289,0.152888,0.00,0.000,inf,0.00000,0.0,0.00000,-0.34,-0.068,-1.764706,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12548,2024-01-14,Eua Ncaa,00:00,San Francisco,Portland,1.06,11.00,143.5,1.80,1.95,-18.5,2.05,1.71,E7lLL4q3,0.943396,0.090909,0.555556,0.512821,0.034305,97.958,8.118068,0.082873,2.4,1.341641,0.559017,97.94,1.306,0.448308,0.343268,83.0,298.148,278.929511,0.935540,0.6,1.341641,2.236068,780.00,5.196,5.597096,1.077193,-65.0,83,52,1.18,15.00,91.802,292.370,1.165612,0.056569,0.127881,1.04,0.208,0.288462,0.00000,0.0,0.00000,-3.17,-0.634,-15.772871,0.000000,0.0,0.000000
12549,2024-01-14,Eua Ncaa,00:00,UC Santa Barbara,Long Beach State,1.80,2.05,156.5,1.80,1.95,-2.5,2.00,1.71,dWJmkWWF,0.555556,0.487805,0.555556,0.512821,0.043360,109.472,20.476116,0.187044,1.2,1.643168,1.369306,82.50,1.490,0.274135,0.183983,-8.0,256.328,271.718032,1.060040,2

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
12405,13:00,Israel Superliga,Maccabi Ironi Ramat Gan,Hapoel Tel-Aviv,2.30,Back Home
12458,11:30,Turquia Tbl,Kapakli Spor,Cayirova,2.22,Back Home
12461,15:30,Itália Liga A,Scafati,Reggiana,1.89,Back Home
12497,21:00,Eua Nba,Minnesota Timberwolves,Los Angeles Clippers,2.18,Back Home
12504,11:30,Alemanha Bbl,Syntainics MBC,Tubingen,1.60,Back Home
12510,15:00,Eua Ncaa,Wisc. Green Bay,Cleveland State,1.87,Back Home
12518,12:00,Europa Liga Letônia,BK Liepaja,Tartu Ulikool,2.13,Back Home
12542,08:00,Espanha Liga Feminina,Jairis F,Ferrol F,1.70,Back Home
12549,00:00,Eua Ncaa,UC Santa Barbara,Long Beach State,1.80,Back Home
